In [31]:
import gzip
from tqdm import tqdm_notebook as tqdm
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp(document)

In [17]:
doc.sents

In [13]:
line.decode("utf-8").split("\t")[2]

'Sturm Ruger & Co. Inc.  (RGR)'

In [45]:
def run_one_file(n):
    with open(f"../data/splitted-tsv/msmarco-docs.tsv.{n:03}", encoding='utf-8') as f, open(f"one-sentence-per-line.{n:03}", 'w', encoding='utf-8') as outf:
        for line in tqdm(f):
            try:
                document = nlp(line.decode("utf-8").split("\t")[3])
                title = line.decode("utf-8").split("\t")[2]
            except:
                document = nlp(line.split("\t")[3])
                title = line.split("\t")[2]
                
            lines_to_write = [title] + [str(sent) for sent in doc.sents]
            outf.writelines(lines_to_write)
            outf.write("\n")

In [46]:
import multiprocessing
from joblib import Parallel, delayed

max_cpus = 20
n_splits = 1000
inputs = tqdm(range(n_splits))

_ = Parallel(n_jobs=max_cpus)(delayed(run_one_file)(i) for i in inputs)

KeyboardInterrupt: 

In [2]:
import torch
from pytorch_pretrained_bert import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel
import logging
logging.basicConfig(level=logging.INFO)
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin not found in cache, downloading to /tmp/tmpfpmwpg2x
100%|██████████| 9143613/9143613 [00:01<00:00, 8667644.21B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpfpmwpg2x to cache at /home/arthur/.pytorch_pretrained_bert/b24cb708726fd43cbf1a382da9ed3908263e4fb8a156f9e0a4f45b7540c69caa.a6a9c41b856e5c31c9f125dd6a7ed4b833fbcefda148b627871d4171b25cffd1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /home/arthur/.pytorch_pretrained_bert/b24cb708726fd43cbf1a382da9ed3908263e4fb8a156f9e0a4f45b7540c69caa.a6a9c41b856e5c31c9f125dd6a7ed4b833fbcefda148b627871d4171b25cffd1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpfpmwpg2x
INFO:pytorch_pretrained_bert.tokenization_transfo_xl:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin from cache at /home/arthur/.pytorch_pretrained_bert/b24

In [3]:
text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"
tokenized_text_1 = tokenizer.tokenize(text_1)
tokenized_text_2 = tokenizer.tokenize(text_2)


In [9]:
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])


In [10]:
model = TransfoXLModel.from_pretrained('transfo-xl-wt103')
model.eval()


INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-pytorch_model.bin not found in cache, downloading to /tmp/tmpndue4eyr
100%|██████████| 1140884800/1140884800 [00:42<00:00, 27148702.66B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpndue4eyr to cache at /home/arthur/.pytorch_pretrained_bert/12642ff7d0279757d8356bfd86a729d9697018a0c93ad042de1d0d2cc17fd57b.e9704971f27275ec067a00a67e6a5f0b05b4306b3f714a96e9f763d8fb612671
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /home/arthur/.pytorch_pretrained_bert/12642ff7d0279757d8356bfd86a729d9697018a0c93ad042de1d0d2cc17fd57b.e9704971f27275ec067a00a67e6a5f0b05b4306b3f714a96e9f763d8fb612671
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpndue4eyr
INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-config.json not found in cache, downloading to /tmp/tmpjx4rdrka
100%|██████████| 606/60

TransfoXLModel(
  (word_emb): AdaptiveEmbedding(
    (emb_layers): ModuleList(
      (0): Embedding(20000, 1024)
      (1): Embedding(20000, 256)
      (2): Embedding(160000, 64)
      (3): Embedding(67735, 16)
    )
    (emb_projs): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 1024x1024]
        (1): Parameter containing: [torch.FloatTensor of size 1024x256]
        (2): Parameter containing: [torch.FloatTensor of size 1024x64]
        (3): Parameter containing: [torch.FloatTensor of size 1024x16]
    )
  )
  (drop): Dropout(p=0.1)
  (layers): ModuleList(
    (0): RelPartialLearnableDecoderLayer(
      (dec_attn): RelPartialLearnableMultiHeadAttn(
        (qkv_net): Linear(in_features=1024, out_features=3072, bias=False)
        (drop): Dropout(p=0.1)
        (dropatt): Dropout(p=0.0)
        (o_net): Linear(in_features=1024, out_features=1024, bias=False)
        (layer_norm): BertLayerNorm()
        (r_net): Linear(in_features=1024, out_features=1024,

In [11]:
tokens_tensor_1 = tokens_tensor_1.to('cuda')
tokens_tensor_2 = tokens_tensor_2.to('cuda')
model.to('cuda')

TransfoXLModel(
  (word_emb): AdaptiveEmbedding(
    (emb_layers): ModuleList(
      (0): Embedding(20000, 1024)
      (1): Embedding(20000, 256)
      (2): Embedding(160000, 64)
      (3): Embedding(67735, 16)
    )
    (emb_projs): ParameterList(
        (0): Parameter containing: [torch.cuda.FloatTensor of size 1024x1024 (GPU 0)]
        (1): Parameter containing: [torch.cuda.FloatTensor of size 1024x256 (GPU 0)]
        (2): Parameter containing: [torch.cuda.FloatTensor of size 1024x64 (GPU 0)]
        (3): Parameter containing: [torch.cuda.FloatTensor of size 1024x16 (GPU 0)]
    )
  )
  (drop): Dropout(p=0.1)
  (layers): ModuleList(
    (0): RelPartialLearnableDecoderLayer(
      (dec_attn): RelPartialLearnableMultiHeadAttn(
        (qkv_net): Linear(in_features=1024, out_features=3072, bias=False)
        (drop): Dropout(p=0.1)
        (dropatt): Dropout(p=0.0)
        (o_net): Linear(in_features=1024, out_features=1024, bias=False)
        (layer_norm): BertLayerNorm()
        

In [12]:
with torch.no_grad():
    # Predict hidden states features for each layer
    hidden_states_1, mems_1 = model(tokens_tensor_1)
    # We can re-use the memory cells in a subsequent call to attend a longer context
    hidden_states_2, mems_2 = model(tokens_tensor_2, mems=mems_1)
